<div class="alert alert-warning">
    
<b>Disclaimer:</b> 
    
The main objective of the <i>Jupyter</i> notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function  
- setting and running the fit  
- extracting and displaying information about the results  

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.  
But, note thatthe initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.

</div> 

# Two Lorentzian &lowast; resolution with lmfit

## Introduction

<div class="alert alert-info">
    
The objective of this notebook is to show how to use one of the models of 
the <a href="https://github.com/QENSlibrary/QENSmodels">QENSlibrary</a>, <b>Lorentzian</b>, to perform some fits.
<a href="https://lmfit.github.io/lmfit-py/">lmfit</a> is used for fitting.
</div>

The following example uses the data from IRIS:
- workspace_index=0, file: `irs26176_graphite002_red.nxs`  
- related instrument resolution data `irs26173_graphite002_res.nxs`   

The ISIS sample datasets can be downloaded from [Mantid's website](http://download.mantidproject.org/).
The data used for this example are in the sample datafile: `data_2lorentzians.dat` and the instrument resolution datafile `irf_iris.dat`, respectively.

This example is based on a [Mantid "Fitting QENS Peaks" tutorial](https://www.mantidproject.org/Fitting_QENS_Peaks).

The implementation with `lmfit` is based on https://lmfit.github.io/lmfit-py/model.html

This example requires an additional Python module `scipy.interpolate` to interpolate the tabulated data of the instrument resolution.

### Physical units
For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

The dictionary of units defined in the cell below specify the units of the refined parameters adapted to the convention used in the experimental datafile.

In [ ]:
# Units of parameters for selected QENS model and experimental data
dict_physical_units = {'omega': "meV", 
                       'q': "1/Angstrom", 
                       'hwhm': "meV", 
                       'scale': "unit_of_signal.meV",
                       'center': "meV"}

## Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
import lmfit
from scipy.interpolate import interp1d
import QENSmodels

path_to_data = './data/'

## Step 1: load instrument resolution data

In [ ]:
irf_iris = np.loadtxt(path_to_data + 'irf_iris.dat')
x_irf = irf_iris[:, 0]
y_irf = irf_iris[:, 1]

## Step 2: create function for instrument resolution data (cubic interpolation between tabulated data points)
Create model: 2 lorentzians convoluted with instrument resolution: 6 parameters

In [ ]:
def irf_gate(x):
    """ 
    Function defined from the interpolation of instrument resolution data 
    Used to define fitting model and plot
    """ 
    f = interp1d(x_irf, y_irf, kind='cubic', bounds_error=False, fill_value='extrapolate')

    return f(x)

# plot tabulated data and interpolated data
xx = np.linspace(-.25, .25, 500)

fig0 = plt.figure()
plt.plot(x_irf, y_irf, 'b.', label='tabulated data')
plt.plot(xx, irf_gate(xx), 'g--', label='extrapolated data')
plt.legend()
plt.xlabel('Energy transfer (meV)')
plt.title('Instrument resolution: plot tabulated data and interpolated data')
plt.grid();

## Step 3: create "double lorentzian" profile

In [ ]:
def model_2lorentzians(x, scale1, center1, hwhm1, scale2, center2, hwhm2):
    return QENSmodels.lorentzian(
        x, 
        scale1, 
        center1, 
        hwhm1
    ) + QENSmodels.lorentzian(
        x, 
        scale2, 
        center2, 
        hwhm2
    )

## Step 4: create convolution function

Code from https://github.com/jmborr/qef

In [ ]:
def convolve(model, resolution):
    c = np.convolve(model, resolution, mode='valid')
    if len(model) % len(resolution) == 0:
        c = c[:-1]
    return c

class Convolve(lmfit.CompositeModel):
    def __init__(self, resolution, model, **kws):
        super(Convolve, self).__init__(resolution, model, convolve, **kws)
        self.resolution = resolution
        self.model = model

    def eval(self, params=None, **kwargs):
        res_data = self.resolution.eval(params=params, **kwargs)
        
        # evaluate model on an extended energy range to avoid boundary effects
        independent_var = self.resolution.independent_vars[0]
        e = kwargs[independent_var]  # energy values
        neg_e = min(e) - np.flip(e[np.where(e > 0)], axis=0)
        pos_e = max(e) - np.flip(e[np.where(e < 0)], axis=0)
        e = np.concatenate((neg_e, e, pos_e))
        kwargs.update({independent_var: e})
        model_data = self.model.eval(params=params, **kwargs)
        
        # Multiply by the X-spacing to preserve normalization
        de = (e[-1] - e[0])/(len(e) - 1)  # energy spacing
        return de * convolve(model_data, res_data)

## Step 5: Load reference data - extract x and y values

In [ ]:
two_lorentzians_iris = np.loadtxt(path_to_data + 'data_2lorentzians.dat')
xx = two_lorentzians_iris[:, 0]
yy = two_lorentzians_iris[:, 1]

In [ ]:
model = Convolve(lmfit.Model(irf_gate), lmfit.Model(model_2lorentzians))

## Step 6: Fit

In [ ]:
result = model.fit(yy, x=xx, scale1=1., center1=0., hwhm1=0.25, scale2=5., center2=0., hwhm2=0.02)

In [ ]:
# plot initial configuration
fig1 = plt.figure()
plt.plot(xx, yy, '+', label='experimental data')
plt.plot(xx, result.init_fit, 'k--', label='initial model')
plt.legend(loc='upper right')
plt.xlabel('Energy transfer (meV)') 
plt.title('Plot before fitting: experimental data and mode with initial guesses')
plt.grid();

## Step 7: Plotting results

In [ ]:
print('Result of fit:\n', result.fit_report())

Plot experimental data and best fit using `matplotlib.pyplot`

In [ ]:
fig2 = plt.figure()
plt.plot(xx, yy, '+', label='experimental data')
plt.plot(xx, result.best_fit, 'r-', label='best fit')
plt.grid()
plt.xlabel('Energy transfer (meV)')
plt.title('Plot selected fitting results: experimental data and best fit')
plt.legend();

Other option: use lmfit features to plot result

In [ ]:
result.plot()

Print values and errors of refined parameters:

In [ ]:
for item in result.params.keys():
    print(f'{item[:-1]}: {result.params[item].value} +/- {result.params[item].stderr} {dict_physical_units[item[:-1]]}')